# 03-02 : Function Hub

Attempt a function from MLRun Function Hub.

## References

- https://github.com/mlrun/mlrun/blob/development/hack/local/mlrun-local.yaml

In [22]:
import os

In [23]:
# set the mlrun api path
server_ip = !(hostname -I | awk '{print $1}')
server_ip = server_ip[0]

os.environ["MLRUN_DBPATH"] = f"http://{server_ip}:8080"

In [24]:
import mlrun

In [25]:
artifact_path = os.path.abspath("jobs")
print(f"Artifacts path: {artifact_path}")

Artifacts path: /home/johnny/swan/code/local-mrlrun/notebooks/03_docker_local/jobs


In [26]:
project = mlrun.get_or_create_project(
    name='hello-function',
    context='./',) 

> 2025-05-12 16:03:56,012 [info] Project loaded successfully: {"project_name":"hello-function"}


In [27]:
project.set_function("hub://describe", "describe")

In [28]:
my_describe = project.get_function("describe")

In [29]:
my_describe.doc()

function: describe
describe and visualizes dataset stats
default handler: analyze
entry points:
  analyze: The function will output the following artifacts per
column within the data frame (based on data types)
If the data has more than 500,000 sample we
sample randomly 500,000 samples:

describe csv
histograms
scatter-2d
violin chart
correlation-matrix chart
correlation-matrix csv
imbalance pie chart
imbalance-weights-vec csv
    context(MLClientCtx)  - The function context
    name(str)  - Key of dataset to database ("dataset" for default), default=dataset
    table(Union[FeatureSet, DataItem])  - MLRun input pointing to pandas dataframe (csv/parquet file path) or FeatureSet as param, default=None
    label_column(str)  - Ground truth column label, default=None
    plots_dest(str)  - Destination folder of summary plots (relative to artifact_path) ("plots" for default), default=plots
    random_state(int)  - When the table has more than 500,000 samples, we sample randomly 500,000 samp

In [30]:
my_describe

In [31]:
!docker ps

767.22s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


CONTAINER ID   IMAGE                                   COMMAND                  CREATED          STATUS                    PORTS                                                                                                                             NAMES
bdf41049b57e   gcr.io/iguazio/alpine:3.20              "/bin/sh -c '/bin/sl…"   32 minutes ago   Up 32 minutes                                                                                                                                               nuclio-local-storage-reader
209105c4fc1d   quay.io/nuclio/dashboard:stable-amd64   "/docker-entrypoint.…"   32 minutes ago   Up 32 minutes (healthy)   80/tcp, 0.0.0.0:8070->8070/tcp, :::8070->8070/tcp                                                                                 03_docker_local-nuclio-1
b8c3292a6246   mlrun/mlrun-ui:1.7.2                    "/docker-entrypoint.…"   32 minutes ago   Up 32 minutes             8080/tcp, 0.0.0.0:8060->8090/tcp, [::]:8060->8090/tcp      

In [38]:
?mlrun.platforms

Type:        module
String form: <module 'mlrun.platforms' from '/home/johnny/swan/miniconda3/envs/local-mlrun/lib/python3.9/site-packages/mlrun/platforms/__init__.py'>
File:        ~/swan/miniconda3/envs/local-mlrun/lib/python3.9/site-packages/mlrun/platforms/__init__.py
Docstring:   <no docstring>

In [39]:
run = my_describe.run(
    name="describe",
    inputs={"table": "https://raw.githubusercontent.com/mlrun/demos/master/iris/iris_dataset.csv"},
    params={"label_column": "label"},
    artifact_path=artifact_path,
    local=True,
)

> 2025-05-13 13:56:51,444 [info] Storing function: {"db":null,"name":"describe","uid":"0937729aba28411badf91bc0155fc094"}
> 2025-05-13 13:56:51,526 [error] Run error, Traceback (most recent call last):
  File "/home/johnny/swan/miniconda3/envs/local-mlrun/lib/python3.9/site-packages/mlrun/runtimes/local.py", line 304, in _run
    fn = self._get_handler(handler, context)
  File "/home/johnny/swan/miniconda3/envs/local-mlrun/lib/python3.9/site-packages/mlrun/runtimes/local.py", line 239, in _get_handler
    return load_module(command, handler, context, embed_in_sys=embed_in_sys)
  File "/home/johnny/swan/miniconda3/envs/local-mlrun/lib/python3.9/site-packages/mlrun/runtimes/local.py", line 408, in load_module
    spec.loader.exec_module(module)
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/tmp/tmpely_63gd.py", line 27, in <module>
    import plotly.express as px
ModuleNotFo

project,uid,iter,start,state,kind,name,labels,inputs,parameters,results
hello-function,...155fc094,0,May 13 11:56:51,error,run,describe,kind=localowner=johnnyhost=dragon,table,label_column=label,


> 2025-05-13 13:56:51,584 [info] Run execution finished: {"name":"describe","status":"error"}


RunError: Failed on pre-loading / post-running of the function, caused by: No module named 'plotly'

In [43]:
!docker ps -a

79634.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


CONTAINER ID   IMAGE                                   COMMAND                  CREATED        STATUS                    PORTS                                                                                                                             NAMES
e64edd4ca671   gcr.io/iguazio/alpine:3.20              "/bin/sh -c '/bin/sl…"   4 hours ago    Up 4 hours                                                                                                                                                  nuclio-local-storage-reader
209105c4fc1d   quay.io/nuclio/dashboard:stable-amd64   "/docker-entrypoint.…"   22 hours ago   Up 22 hours (healthy)     80/tcp, 0.0.0.0:8070->8070/tcp, :::8070->8070/tcp                                                                                 03_docker_local-nuclio-1
b8c3292a6246   mlrun/mlrun-ui:1.7.2                    "/docker-entrypoint.…"   22 hours ago   Up 22 hours               8080/tcp, 0.0.0.0:8060->8090/tcp, [::]:8060->8090/tcp              

In [46]:
?project.build_function

Signature:
project.build_function(
    function: Union[str, mlrun.runtimes.base.BaseRuntime],
    with_mlrun: bool = None,
    skip_deployed: bool = False,
    image: str = None,
    base_image: str = None,
    commands: list = None,
    secret_name: str = None,
    requirements: Union[str, list[str]] = None,
    mlrun_version_specifier: str = None,
    builder_env: dict = None,
    overwrite_build_params: bool = False,
    requirements_file: str = None,
    extra_args: str = None,
    force_build: bool = False,
) -> Union[mlrun.projects.operations.BuildStatus, kfp.dsl._container_op.ContainerOp]
Docstring:
deploy ML function, build container with its dependencies

:param function:            name of the function (in the project) or function object
:param with_mlrun:          add the current mlrun package to the container build
:param skip_deployed:       skip the build if we already have an image for the function
:param image:               target image name/path
:param base_image:    

In [49]:
result = project.build_function("describe")
result

BuildStatus(ready=True, outputs={'image': 'mlrun/mlrun'})

In [51]:
project.build_image()

> 2025-05-13 14:04:11,255 [info] Base image not specified, using default base image: {"base_image":"mlrun/mlrun"}
> 2025-05-13 14:04:11,287 [error] error submitting build task: details: {'reason': 'Runtime error: Default docker registry is not defined, set MLRUN_HTTPDB__BUILDER__DOCKER_REGISTRY/MLRUN_HTTPDB__BUILDER__DOCKER_REGISTRY_SECRET env vars'}, caused by: 400 Client Error: Bad Request for url: http://192.168.1.184:8080/api/v1/build/function


OSError: error: cannot submit build, details: {'reason': 'Runtime error: Default docker registry is not defined, set MLRUN_HTTPDB__BUILDER__DOCKER_REGISTRY/MLRUN_HTTPDB__BUILDER__DOCKER_REGISTRY_SECRET env vars'}, caused by: 400 Client Error: Bad Request for url: http://192.168.1.184:8080/api/v1/build/function

In [50]:
!docker ps -a

79948.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


CONTAINER ID   IMAGE                                   COMMAND                  CREATED        STATUS                    PORTS                                                                                                                             NAMES
e64edd4ca671   gcr.io/iguazio/alpine:3.20              "/bin/sh -c '/bin/sl…"   5 hours ago    Up 5 hours                                                                                                                                                  nuclio-local-storage-reader
209105c4fc1d   quay.io/nuclio/dashboard:stable-amd64   "/docker-entrypoint.…"   23 hours ago   Up 23 hours (healthy)     80/tcp, 0.0.0.0:8070->8070/tcp, :::8070->8070/tcp                                                                                 03_docker_local-nuclio-1
b8c3292a6246   mlrun/mlrun-ui:1.7.2                    "/docker-entrypoint.…"   23 hours ago   Up 23 hours               8080/tcp, 0.0.0.0:8060->8090/tcp, [::]:8060->8090/tcp              